In [1]:
import sys
import os
sys.path.append(os.path.abspath('../..'))

# Generating data

All the code for data generation is in the module `generate_lib`.

The quantum chemistry data used in this project are produced using the quantum chemistry packagePsi4 (http://www.psicode.org/).
Thanks to the interface package `openfermionpsi4`, the resulting molecular data can be stored in a molecule `.hdf5` file, to be directly loaded inside an `openfermion.MolecularData` object.

The molecule files are stored in this repository's `data/molecules/` directory.
The name of each file is determined by univocally converting the molecule's geometry to a string (see `MoleculeDataGenerator._generate_filename`).

For each molecule, the relevant data for the ML model are also saved as a dictionary in a `.json` with the same name, in the structured directory `data/json/`.
The data in the json files is accessible and usable without dependency on any of the aforementioned packages.

To generate and save a molecule and relative data, given the geometry, it is sufficient to instantiate the object `MoleculeDataGenerator(geometry)`.

In [2]:
from convoQC.utils.generate_lib import MoleculeDataGenerator

# Set the geometry and generate the molecule

geometry = (
    ('H', (0., 0., 0.)),
    ('H', (1., 0., 0.)),
    ('H', (0., 1., 0.)),
    ('H', (0., 0., 1.))
)

# instantiating a MoleculeDataGenerator is enough to create molecule+data files
gen = MoleculeDataGenerator(geometry) 

# the object contains the molecule and the relative data dictionary
print(gen.molecule)
print(gen.data_dict.keys())

dict_keys(['geometry', 'multiplicity', 'canonical_orbitals', 'canonical_to_oao', 'orbital_energies', 'exact_energy', 'ground_states', 'hf_energy'])


In [3]:
print(gen.filename)

H,0,0,0;H,1,0,0;H,0,1,0;H,0,0,1


Check that degenerate ground state subspace basis vectors are orthonormal

In [4]:
import numpy as np
from itertools import combinations_with_replacement
ground_states = gen.data_dict['ground_states']
for i, j in combinations_with_replacement(range(3), 2):
    print(f'|<{i}|{j}>|^2 = ', 
          round(np.abs(ground_states[:,i].conj() @ ground_states[:,j])**2, 12))

|<0|0>|^2 =  1.0
|<0|1>|^2 =  0.0
|<0|2>|^2 =  0.0
|<1|1>|^2 =  1.0
|<1|2>|^2 =  0.0
|<2|2>|^2 =  1.0


Activate and use the following cell to erase the `H,0,0,0;H,1,0,0;H,0,1,0;H,0,0,1` files and test molecule and data generation.

## Chosen molecule family

The first chosen molecule family for this project is $\mathrm{H}_4$ in various geometries.
Some physical limits for the geometry are set:
- For each pair of H atoms, the interatomic distance is not smaller than $0.4Å$. This avoids exaggerate orbital ovelaps
- For each pair of adjacent atoms (in the ordered in which they're listed in the geometry) the interatomic distance is no more than $1.5Å$. This avoids completely dissociated molecules.

Additionally, parameters that are irrelevant for the calculation of translation-invariant and rotation-invariant properties are (for now) fixed:
- the fist atom is always at position $(0, 0, 0)$
- the second atom is always on the positive X half-axis $(x_1, 0, 0)$, $x_1>0$
- the third atom is always on the XY plane $(x_2, y_2, 0)$
- the fourth can be anywhere in space, within the previously set limits $(x_3, y_3, z_3)$

Finally, for convenience in file naming and data exchange, we keep only 4 decimals in all the $x_i, y_i, z_i$ values:
- all positions are forced on a grid with resolution $0.001Å$

In [5]:
from convoQC.utils import H4_generate_random_molecule
help(H4_generate_random_molecule)

Help on function H4_generate_random_molecule in module convoQC.utils.generate_lib:

H4_generate_random_molecule(rng: numpy.random._generator.Generator = Generator(PCG64) at 0x7FAFC4717550, dmin: float = 0.4, dmax: float = 1.5, rounding: int = 4)
    Generate and save molecule and data for a valid random geomertry of H4.
    
    The molecule geometry is of the form:
    (
        ('H', (0 , 0 , 0 )),
        ('H', (x1, 0 , 0 )),
        ('H', (x2, y2, 0 )),
        ('H', (x3, y3, z3))
    )
    with the additional constraints:
        - `dmin` minimum distance of atom pairs
        - `dmax` maximum distance of adjacent atoms
    and all values rounded to `rounding` digits
    
    Args:
        rng: a numpy.random generator
        dmin: minimum distance between each pair of atoms
        dmax: maximum distance between adjacent atoms
        rounding: number of digits to which all positions are rounded
    
    Returns:
        MoleculeDataGenerator
    
    Raises:
        FailedGener

In [6]:
from convoQC.utils.generate_lib import H4_generate_valid_geometry
H4_generate_valid_geometry()

[('H', (0.0, 0.0, 0.0)),
 ('H', (0.8245, 0.0, 0.0)),
 ('H', (-0.6285, 0.0549, 0.0)),
 ('H', (0.4455, 0.1899, 0.0464))]

## Generate random H4 data

In [7]:
from tqdm import tqdm
from convoQC.utils import H4_generate_random_molecule, FailedGeneration

n_molecules_to_generate = 500

for _ in tqdm(range(n_molecules_to_generate)):
    for attempt in range(10):
        try:
            H4_generate_random_molecule()
        except FailedGeneration as exc:
            print('Failed to generate random molecule because of:\n' + str(exc))
        else:
            break

100%|██████████| 500/500 [26:18<00:00,  3.16s/it]


# Loading data for QML model

Only the function `load_data` in `load_lib` is needed to load the relevant data for the QML model.
`load_lib` also defined `JSON_DIR` and `MOLECULES_DIR` for convenience.


In [8]:
from convoQC.utils import load_data, JSON_DIR

To load all data:

In [9]:
dataset = [load_data(filename) 
           for filename in os.listdir(JSON_DIR)
           if filename.endswith('.json')]

print('length of the dataset:', len(dataset))

length of the dataset: 501


**Example:** count how many of the molecules in the dataset have a singlet ground state and how many have a triplet

In [10]:
multiplicities = [load_data(filename)['multiplicity']
                  for filename in os.listdir(JSON_DIR)
                  if filename.endswith('.json')]

from collections import Counter
count = dict(Counter(multiplicities))
length = len(multiplicities)
for k, v in count.items():
    print(f'{v/length*100:0.1f}% with multiplicity {k} ({v}/{length})')

89.6% with multiplicity 1 (449/501)
10.4% with multiplicity 3 (52/501)


In [11]:
hf_errors = [d['hf_energy'] - d['exact_energy'] for d in dataset]
print(f'Average HF error: {np.mean(hf_errors):.3f} '
      f'with stddev {np.std(hf_errors):.3f}')

Average HF error: 0.051 with stddev 0.030


# What are the saved data 

Let's take as an example one data dictionary:

In [12]:
from convoQC.utils import load_data
import os

filename = 'H,0,0,0;H,1,0,0;H,0,1,0;H,0,0,1'
data_dict = load_data(filename)

print('\ncontent of each data dictionary\n')
print(f"{'KEY:':20} {'VALUE TYPE:':20}\n{'-'*60}")
for k, v in data_dict.items():
    print(f'{k:20} {str(type(v)):20}',
          f'with shape {v.shape}' if isinstance(v, np.ndarray) else "")


content of each data dictionary

KEY:                 VALUE TYPE:         
------------------------------------------------------------
geometry             <class 'list'>       
multiplicity         <class 'int'>        
canonical_orbitals   <class 'numpy.ndarray'> with shape (4, 4)
canonical_to_oao     <class 'numpy.ndarray'> with shape (4, 4)
orbital_energies     <class 'numpy.ndarray'> with shape (4,)
exact_energy         <class 'float'>      
ground_states        <class 'numpy.ndarray'> with shape (256, 3)
hf_energy            <class 'float'>      


## Input

`geometry` is a list of tuples ('atom_symbol', (x, y, z)):

In [49]:
data_dict['geometry']

[['H', [0.0, 0.0, 0.0]],
 ['H', [1.0, 0.0, 0.0]],
 ['H', [0.0, 1.0, 0.0]],
 ['H', [0.0, 0.0, 1.0]]]

`multiplicity` indicates wether the ground state of this molecule is a singlet (1) or triplet (3).
All the ground states are saved as complex **column vectors** in a matrix of shape ($2^n$, `multiplicity` )

In [50]:
print('multiplicity: ', data_dict['multiplicity'])
print('ground states: \n', data_dict['ground_states'].round(3))

multiplicity:  3
ground states: 
 [[ 0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j]
 [-0.089+0.j -0.   +0.j  0.001+0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   

## output

`exact_energy` encodes the exact ground-state energy of the molecule, i.e. the target output of the QML model for this first stage of the project.

In [13]:
data_dict['exact_energy']

-1.8917479285527508

## benchmarking 

`hf_energy` stores the Hartree-Fock (HF) energy, i.e. the minimum energy of any single slater-determinant state, under mean field interactions.
This is the quantity that is minimized by the HF variational self-consistent mean field method.
Hartree-Fock is the first, "roughest" estimate of the ground state energy provided by quantum chemistry calculations.
The zeroth requirement of a post-HF method for esrimating the ground state energy, is for its result to be more precise than the HF energy.

In [14]:
data_dict['hf_energy']

-1.8519449915291792

## Atomic and molecular orbitals (eventual additional input)

Atomic orbitals are a physically-motivated, non-complete and non-orthogonal parametrization of scalar funcions of space (fields $\phi(\vec{x})$).

To each atom we can assign a set of spherical harmonics centered in the atom's position and with radii dependent on the atom's charge. 
Each (infinite) set of spherical harmonics forms a orthonormal base for fields.
If we model an atom as a point charge, these spherical harmonics will be the eigen-wavefunctions for a single electron in the atom's field.
Truncating the number of spherical harmonics allows us to construct a *finite* parametrization of space that can approximate the low-energy electron field, hopefully also in the interacting and poly-atomic case.
The basis functions of these parametrization are called atomic orbitals. 

The *minimal basis* approximation, in the case of Hydrogen atoms, prescribes that we take a single spherically symmetric *atomic orbital* for each Hydrogen - to which we have to add spin information.
This results in two *spin-orbitals* per each Hydrogen atom: a total of 4 atomic orbitals (i.e. 8 spin-orbitals) for an H4 molecule.

To construct an orthonormal (of course still incomplete) parametrization of fields, we take linear combinations of these 4 orbitals. The *canonical orbitals* a a special orthonormal combination of atomic orbitals, obtained through the Hartree-Fock method.
The ground-state Slater determinant constructed on the canonical orbitals minimizes the total energy, accounting for electron-electron interactions with a mean-field approach.

The `canonical_orbitals` matrix encodes the linear combination of atomic orbitals taken to construct the canonical orbitals:

In [15]:
data_dict['canonical_orbitals'].round(3)

array([[ 0.474, -0.   ,  0.   ,  1.287],
       [ 0.282,  0.105,  0.963, -0.563],
       [ 0.282, -0.887, -0.391, -0.563],
       [ 0.282,  0.782, -0.572, -0.563]])

The rows relate to each atomic orbital, ordered as the respective atoms appear in `geometry`.
The columns represent molecular orbitals, ordered by increasing single-particle energy.
These are the energies saved in `orbital_energies`.

In [16]:
data_dict['orbital_energies'].round(3)

array([-0.682,  0.043,  0.043,  0.569])

Typically on a quantum computer, under Jordan-Wigner encoding, each pair of qubits will represent a canonical orbital (2 qubits because for each orbital there are two spins, i.e. two spin-orbitals).
The `ground_states` saved in these data are encoded in this way.

The `canonical_to_oao` unitary matrix encodes which linear combination of atomic orbitals needs to be taken to construct a orthonormal version of the atomic orbitals (Orthogonal Atomic Orbitals, OAO).
Columns encode the OAOs shape in the canonical orbital basis.

This might be useful in later stages of the project: using a Givens rotations circuit we can change the state encoding such that each qubit corresponds to one spin-OAO.
This would allow to directly connect the quantum state to the geometry, as each orbital would be "localized" at the position of the respective atom.

In [17]:
data_dict['canonical_to_oao'].round(3)

array([[ 0.632,  0.447,  0.447,  0.447],
       [-0.   ,  0.088, -0.747,  0.659],
       [ 0.   ,  0.812, -0.329, -0.482],
       [ 0.775, -0.365, -0.365, -0.365]])

### details on OAO
Some details about the symmetric orthogonalization procedure (Lödwin's method) used to obtain the orthogonal atomic orbitals.

The `canonical_orbitals` the matrix $M$ represents, as column vectors, the *canonical orbitals* in the (nonorthogonal) basis of the *atomic orbitals*.
The inverse, $P$, transforms a column vector from the atomic orbital basis to the canonical orbitals basis.

As canonical orbitals are orthogonal, in their basis inner product is simply vector dot product: we use the transformation $P$ to calculate the inner product of the atomic orbital basis functions. With this technique we compute the atomic orbital overlap matrix $S$.

Lödwin's method prescribes to use the inverse of the Hermitian square root of the overlap, $C=\sqrt{S^{-1}}$ to orthonormalize the atomic orbital functions, obtaining the ***orthogonal atomic orbitals (OAO)***.
Clearly, the resulting basis is orthonormal as

$$
\langle \mathrm{OAO}_i \vert \mathrm{OAO}_j \rangle 
= C_{il} \langle \mathrm{AO}_l \vert \mathrm{AO}_k \rangle C_{kj}
= ( C \cdot S \cdot C )_{ij} = (C \cdot C^{-2} \cdot C)_{ij} = \delta_{ij}
$$

Finally, combining $C$ and $M$, we get the unitary matrix representing the canonical orbitals in the orthogonal atomic orbital basis, and its inverse.
This last one, is the one saved as `canonical_to_oao` in the data dictionary


In [18]:
from scipy.linalg import sqrtm, inv

M = data_dict['canonical_orbitals']
print('M (CO column vectors in the AO basis)\n', M.round(3))
print()
P = inv(M)
print('P = M^-1 (AO column vectors in the CO basis)\n', P.round(3)) # CO to AO
print()
S = P.T @ P
print('S (overlap matrix of AOs)\n', S.round(5))
print()
C = inv(sqrtm(S))
print('C (OAO column vectors in AO basis)\n', C.round(3))
print()
co_to_oao = (sqrtm(P.T @ P) @ M)
print('U (CO column vectors in OAO basis)\n', 
      co_to_oao.round(3)) # CO from OAO
print()
oao_to_co = (P @ sqrtm(M @ M.T)) # == P @ C
print('U^dag == `canonical_to_oao` (OAO column vectors in CO basis)\n', 
      oao_to_co.round(3)) # OAO from CO

M (CO column vectors in the AO basis)
 [[ 0.474 -0.     0.     1.287]
 [ 0.282  0.105  0.963 -0.563]
 [ 0.282 -0.887 -0.391 -0.563]
 [ 0.282  0.782 -0.572 -0.563]]

P = M^-1 (AO column vectors in the CO basis)
 [[ 0.894  0.681  0.681  0.681]
 [ 0.     0.074 -0.63   0.555]
 [ 0.     0.684 -0.278 -0.406]
 [ 0.448 -0.251 -0.251 -0.251]]

S (overlap matrix of AOs)
 [[1.      0.49648 0.49648 0.49648]
 [0.49648 1.      0.2899  0.2899 ]
 [0.49648 0.2899  1.      0.2899 ]
 [0.49648 0.2899  0.2899  1.     ]]

C (OAO column vectors in AO basis)
 [[ 1.296 -0.258 -0.258 -0.258]
 [-0.258  1.123 -0.064 -0.064]
 [-0.258 -0.064  1.123 -0.064]
 [-0.258 -0.064 -0.064  1.123]]

U (CO column vectors in OAO basis)
 [[ 0.632 -0.     0.     0.775]
 [ 0.447  0.088  0.812 -0.365]
 [ 0.447 -0.747 -0.329 -0.365]
 [ 0.447  0.659 -0.482 -0.365]]

U^dag == `canonical_to_oao` (OAO column vectors in CO basis)
 [[ 0.632  0.447  0.447  0.447]
 [-0.     0.088 -0.747  0.659]
 [ 0.     0.812 -0.329 -0.482]
 [ 0.775 -0.365

# Utilities

## List  files

In [21]:
import os
from convoQC.utils import MOLECULES_DIR, JSON_DIR
    
molecule_files = sorted(os.listdir(MOLECULES_DIR))
data_files = sorted(os.listdir(JSON_DIR))
    
print(f'MOLECULES_DIR content: {len(molecule_files)} files')
print(*molecule_files[:5], '...' if len(molecule_files)>5 else '', sep='\n')
print(f'\nDATA_DIR content: {len(data_files)} files')
print(*data_files[:5],  '...' if len(data_files)>5 else '', sep='\n')

MOLECULES_DIR content: 501 files
H,0,0,0;H,0.401,0,0;H,-0.1196,0.9943,0;H,0.0526,0.2628,-0.3113.hdf5
H,0,0,0;H,0.4023,0,0;H,0.3393,1.0607,0;H,-0.0812,-0.0573,-0.426.hdf5
H,0,0,0;H,0.4028,0,0;H,-0.4404,0.4811,0;H,0.0828,-0.7849,-0.2394.hdf5
H,0,0,0;H,0.4059,0,0;H,0.3759,1.4515,0;H,0.4392,1.1007,0.7042.hdf5
H,0,0,0;H,0.4069,0,0;H,-0.3451,-0.2725,0;H,-1.0738,-0.6908,-0.3546.hdf5
...

DATA_DIR content: 501 files
H,0,0,0;H,0.401,0,0;H,-0.1196,0.9943,0;H,0.0526,0.2628,-0.3113.json
H,0,0,0;H,0.4023,0,0;H,0.3393,1.0607,0;H,-0.0812,-0.0573,-0.426.json
H,0,0,0;H,0.4028,0,0;H,-0.4404,0.4811,0;H,0.0828,-0.7849,-0.2394.json
H,0,0,0;H,0.4059,0,0;H,0.3759,1.4515,0;H,0.4392,1.1007,0.7042.json
H,0,0,0;H,0.4069,0,0;H,-0.3451,-0.2725,0;H,-1.0738,-0.6908,-0.3546.json
...


## Prompt to delete molecule and data files (CAUTION)